# Orbital Parameters Calculator

Get the parameters for an elliptical orbit.

## Compute Internal Units

Get the units for the simulation.

In [28]:
GRAVITY = 6.6743e-8 # 6.6743 × 10-8 cm3 g-1 s-2 (cgs)
LENGTH_UNIT_CM =  3.085678e21 # 1 kpc / h
MASS_UNIT_G = 1.989e43 # 10^10 M_sun /  h
VELOCITY_UNIT_CM_S = 1.0e5 # 1 km / s
TIME_UNIT_S = LENGTH_UNIT_CM / VELOCITY_UNIT_CM_S
INTERNAL_G = (GRAVITY / LENGTH_UNIT_CM**3) * MASS_UNIT_G * TIME_UNIT_S**2

## Set Orbital Parameters

Specify the orbit.

In [49]:
import numpy as np
radius = 100 # 100 kpc / h
m1 = 10
m2 = 10

x1 = np.array([radius/2, 0., 0.])
x2 = np.array([-radius/2, 0., 0.])
r_hat = x2 - x1

g_acc_1 = r_hat * (INTERNAL_G * m2) / np.linalg.norm(r_hat)**3

# g_acc = v**2 / r
# v_circ_1 = (np.linalg.norm(g_acc_1) * radius)**0.5
v_circ_1 = (INTERNAL_G * m1 / (2*radius))**0.5
print(r_hat, g_acc_1, v_circ_1, v_circ_1 * VELOCITY_UNIT_CM_S / (LENGTH_UNIT_CM / TIME_UNIT_S))

[-100.    0.    0.] [-43.02193132   0.           0.        ] 46.379915546878124 46.37991554687812


In [38]:
period = 2. * np.pi * radius / v_circ_1
period

np.float64(9.579325200943407)